In [1]:
import pickle
import pandas as pd
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import glob

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Block distribution by recipe in FutureFoam

![asd](img/block_distribution_by_recipe.JPG)

In [3]:
# foam_types = [i.split('/')[1].split('-')[0] for i in glob.glob('models/*')]
with open('block_dist.dat','r') as inf:
    block_dist = eval(inf.read())

foam_types = list(block_dist.keys())

In [4]:
# adding the foam_type dropdown
foam_type_dropdown = widgets.Dropdown(
    options=foam_types,
    description='Foam type:',
    disabled=False,
    readout=True
)

In [5]:
with open('data_dict.dat','r') as inf:
    dict_from_file = eval(inf.read())

In [6]:
good_dict = {}
for key, value in dict_from_file.items():
    good_dict[key] = pd.DataFrame(value)

In [7]:
# good_dict['60135']
dict_recipes = {}
dict_recipes = good_dict

In [8]:
def draw_GBR_shrinkage(fo_type):
    # leave in the X predictor variables only the curing time
    X = dict_recipes[fo_type][['curing_time']]
    # we are trying to predict Height shrinkage
    Y = dict_recipes[fo_type]['FILT_HEIGHTA_SHRINKAGE']
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)
        
    filename = fo_type + '-boosting_regression_model.pkl'
    path = 'models/'
    loaded_model = pickle.load(open(path + filename, 'rb'))
      
    loaded_model.fit(x_train, y_train)
    print("Accuracy of: %f%% " % (loaded_model.score(x_test, y_test)*100))
    
    plt.figure(figsize=(20,5))
    plt.scatter(X, Y,color='r')
    plt.plot(X, loaded_model.predict(X),color='k')
    plt.xlabel('curing time [h]')
    plt.ylabel('Height shrinkage [mm]')
    # plt.legend(['Gradient boosting regression', 'Actual shrinkage'])
    plt.legend(['Prediction', 'Actual shrinkage'])
    plt.grid()
    plt.show()
    
    horas_test = [12, 24, 48, 72, 96, 120, 200, 400]
    prueba_manual = pd.DataFrame(data={'curing_time':horas_test})
    pred_result = loaded_model.predict(prueba_manual)
    
    print("Recipe %s has %d blocks" % (fo_type, block_dist[fo_type]))
    
    for pred,a in zip(pred_result, horas_test):
        print('Prediction for %dh: %f mm' % (a, pred))

## Shrinkage prediction based on curing time


In [9]:
interact(draw_GBR_shrinkage, fo_type=foam_type_dropdown);

interactive(children=(Dropdown(description='Foam type:', options=('36125', '60135', '30100', '85150C', '18110'…